In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from src.models.load_data import Balance, Instruments, AdvisedPortfolios, PriceDB, Singleton

In [2]:
access_path = './data/processed/'
bal_from_nm = 'bal_btw_rebal_20210602.dat'

In [3]:
bal_from = pd.read_csv(access_path + bal_from_nm)
price_db = PriceDB.instance().data

In [4]:
last_dt = bal_from.iloc[-1,0]

In [5]:
bal_from = bal_from[bal_from.date==last_dt]

#### MP 발생일
- 2021.5.3
- 2021.6.2
- 2021.7.2

In [6]:
# 두 MP발생일 사이 기간 설정
start_dt = '20210603'  # MP발생 다음날부터
end_dt = '20210701'    # MP발생 전날까지

range_dt_ymd = pd.date_range(start=start_dt, end=end_dt, freq='D')  # price_db 날짜 포맷
range_dt_mdy = range_dt_ymd.strftime('%-m/%-d/%Y')  # 잔고(balance) 날짜 포맷

#### 최근 과거 MP발생일 잔고를 해당 기간 내 매일 매일의 잔고로 복사해 두고, price 컬럼과 value 컬럼을 업데이트하면 끝.

- 잔고 복사

In [7]:
df_list = []
for cnt, dt in enumerate(range_dt_ymd):
    temp = bal_from.copy(deep=True)
    temp.date = dt
    df_list.append(temp)
    
bal_btw_rebal = pd.concat(df_list)

- `price_db`를 이용해서 Price 컬럼 업데이트.

In [8]:
bal_btw_rebal = bal_btw_rebal.set_index(['date', 'itemcode'])
price_db = price_db.set_index(['date', 'itemcode'])
bal_btw_rebal.update(price_db)
bal_btw_rebal = bal_btw_rebal.reset_index(drop=False)

- 사용자별/날짜별로 비중(`wt`) 계산

In [9]:
bal_btw_rebal.value = bal_btw_rebal.quantity*bal_btw_rebal.price
bal_btw_rebal.original = ''
bal_btw_rebal.wt = bal_btw_rebal.value/bal_btw_rebal.groupby(['date', 'userid'])['value'].transform('sum')

In [10]:
bal_btw_rebal

,date,itemcode,userid,name,asset_class,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,original
0,2021-06-03,A114260,A001,안정추구형소규모,Fixed Income,KODEX 국고채3년,2,NaN,NaN,58020.0,116040.0,0.165477,NaN,
1,2021-06-03,A273130,A001,안정추구형소규모,Fixed Income,KODEX 종합채권(AA-이상)액티브,0,NaN,NaN,108325.0,0.0,0.000000,NaN,
2,2021-06-03,A196230,A001,안정추구형소규모,Fixed Income,KBSTAR 단기통안채,1,NaN,NaN,104480.0,104480.0,0.148992,NaN,
3,2021-06-03,A122260,A001,안정추구형소규모,Fixed Income,KOSEF 통안채1년,2,NaN,NaN,101275.0,202550.0,0.288843,NaN,
4,2021-06-03,A130730,A001,안정추구형소규모,Fixed Income,KOSEF 단기자금,2,NaN,NaN,101130.0,202260.0,0.288430,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2257,2021-07-01,A278540,A009,적극투자형대규모,Equity,KODEX MSCI Korea TR,23,NaN,NaN,14010.0,322230.0,0.235689,NaN,
2258,2021-07-01,A266370,A009,적극투자형대규모,Equity,KODEX IT,8,NaN,NaN,21175.0,169400.0,0.123904,NaN,
2259,2021-07-01,A114260,A009,적극투자형대규모,Fixed Income,KODEX 국고채3년,2,NaN,NaN,57845.0,115690.0,0.084619,NaN,
2260,2021-07-01,C000001,A009,적극투자형대규모,Cash,현금,80,NaN,NaN,1.0,80.0,0.000059,NaN,


In [11]:
bal_btw_rebal.to_csv(access_path + 'bal_btw_rebal.dat', index=False, encoding='utf-8-sig')

In [12]:
bal_btw_rebal.columns

Index(['date', 'itemcode', 'userid', 'name', 'asset_class', 'itemname',
       'quantity', 'cost_price', 'cost_value', 'price', 'value', 'wt',
       'group_by', 'original'],
      dtype='object')